In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/bond_etf_yield.csv', index_col=0)
df

,Name,Yield to Maturity,Date
Ticker,,,
AGG,iShares Core U.S. Aggregate Bond ETF,0.0502,08-11-2023
TLT,iShares 20+ Year Treasury Bond ETF,0.0439,08-11-2023
LQD,iShares iBoxx $ Investment Grade Corporate Bon...,0.0562,08-11-2023
MUB,iShares National Muni Bond ETF,0.0346,08-11-2023
IEF,iShares 7-10 Year Treasury Bond ETF,0.0420,08-11-2023
...,...,...,...
MBSD,FLEXSHARES DISCIPLINED DURATION MBS INDEX FUND,0.0540,08-11-2023
RAVI,FLEXSHARES ULTRA-SHORT INCOME FUND,0.0581,08-11-2023
SKOR,FLEXSHARES CREDIT-SCORED US CORPORATE BOND IND...,0.0574,08-11-2023


In [3]:
df.sort_values(by='Yield to Maturity', ascending=False).dropna()

,Name,Yield to Maturity,Date
Ticker,,,
PGHY,Invesco Global Short Term High Yield Bond ETF,0.1022,08-11-2023
JBBB,B-BBB CLO ETF,0.1001,08-11-2023
HYGV,FLEXSHARES HIGH YIELD VALUE-SCORED BOND INDEX ...,0.0961,08-11-2023
BRLN,BlackRock Floating Rate Loan ETF,0.0948,08-11-2023
EMHY,iShares J.P. Morgan EM High Yield Bond ETF,0.0948,08-11-2023
...,...,...,...
BBIP,JPMorgan BetaBuilders U.S. TIPS 0-5 Year ETF,0.0254,07-31-2023
TIP,iShares TIPS Bond ETF,0.0233,08-11-2023
PBTP,Invesco PureBeta℠ 0-5 Yr US TIPS ETF,0.0218,08-11-2023


In [4]:
df.loc['BND']

Name                 Vanguard Total Bond Market ETF
Yield to Maturity                             0.048
Date                                     06-30-2023
Name: BND, dtype: object